## LiDAR Projection Test

In [35]:
from nuscenes.nuscenes import NuScenes
from pyquaternion.quaternion import Quaternion
from nuscenes.utils.geometry_utils import view_points, box_in_image, BoxVisibility, transform_matrix
from nuscenes.utils.data_classes import PointCloud, LidarPointCloud, RadarPointCloud, Box
import numpy as np
import shutil
from PIL import Image
import os.path as osp
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
from matplotlib import cm
from sphere import Sphere
from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map
from functools import partial
from dh_grid import DHGrid
from sphere import Sphere
from img_sphere import ImageSphere, ColorImageSphere
from metrics import *
from average_meter import AverageMeter
from utils import Utils

%matplotlib inline
%load_ext autoreload
%autoreload 2


dataset_path = '/mnt/data/datasets/nuscenes/v1.0-mini/'

nusc = NuScenes(version='v1.0-mini', dataroot=dataset_path, verbose=True)
all_cam_strings = ['CAM_FRONT','CAM_FRONT_RIGHT','CAM_BACK_RIGHT','CAM_BACK','CAM_BACK_LEFT','CAM_FRONT_LEFT']
lidar_string = 'LIDAR_TOP'
bw = 200

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loading NuScenes tables for version v1.0-mini...
Loading nuScenes-lidarseg...
32 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
404 lidarseg,
Done loading in 0.327 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [36]:
def get_semantic_point_cloud_from_token(sample_token):
    sample = nusc.get('sample', sample_token)
    pointsensor = nusc.get('sample_data', sample['data'][lidar_string])
    # Load a single lidar point cloud.
    pcl_path = osp.join(dataset_path, pointsensor['filename'])
    pc = LidarPointCloud.from_file(pcl_path)
    points_xyzi = pc.points.T
    
    # Load the semantic segmentation.
    sample_data_token = sample['data'][lidar_string]
    lidarseg_labels_filename = osp.join(nusc.dataroot, nusc.get('lidarseg', sample_data_token)['filename'])
    points_label = np.fromfile(lidarseg_labels_filename, dtype=np.uint8)  # [num_points]

    # Combine the two.
    return np.column_stack((points_xyzi, points_label))

def mapIntensityToRGB(i):
    mask = np.where(i < 0)    
    colors = cm.jet(plt.Normalize(min(i), max(i))(i))
    colors[mask] = 0
    return colors

def prepare_for_viz(cloud):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    if cloud.shape[1] == 4:
        colors = mapIntensityToRGB(cloud[:, 3])
        pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    if cloud.shape[1] == 6:
        pcd.colors = o3d.utility.Vector3dVector(cloud[:,3:6] / 255.0)
    return pcd

def visualize_pointcloud(cloud):
    pcd = prepare_for_viz(cloud)        
    o3d.visualization.draw_geometries([pcd])
    
def visualize_sphere(cloud, points):    
    cur_sem_cloud = np.reshape(cloud, (3, -1)).T
    points_xyz = DHGrid.ConvertPointsToEuclidean(points)        
    points_xyzi = np.column_stack((points_xyz, cur_sem_cloud[:,0]))
    visualize_pointcloud(points_xyzi)

In [53]:
scene = nusc.scene[0]
first_sample_token = scene['first_sample_token']
pc = get_semantic_point_cloud_from_token(first_sample_token)
visualize_pointcloud(pc)

In [58]:
grid, points = DHGrid.CreateGrid(bw)
sample_sphere = Sphere(pc)
feature = sample_sphere.sampleUsingGrid(grid)
visualize_sphere(feature, points)

In [62]:
# visualize_pointcloud(points)
DHGrid.ConvertPointsToEuclidean(points).shape
xyz_grid = DHGrid.ConvertGridToEuclidean(grid)

xyz_grid = np.reshape(xyz_grid, (3, -1)).T
xyz_feature = np.reshape(feature, (3, -1)).T
visualize_pointcloud(np.column_stack((xyz_grid, xyz_feature[:,0])))

In [67]:
def __projectPointCloudOnSphere(cloud):
    # sqrt(x^2+y^2+z^2)
    dist = np.sqrt(cloud[:,0]**2 + cloud[:,1]**2 + cloud[:,2]**2)
    #dist = np.sqrt(np.power(sph_image_cart[:,0],2) + np.power(sph_image_cart[:,1],2) + np.power(sph_image_cart[:,2],2))

    projected = np.empty([len(cloud), 2])
    ranges = np.empty([len(cloud), 1])

    # Some values might be zero or NaN, lets ignore them for now.
    eps = 0.000001
    projected[:,0] = np.arccos(cloud[:,2] / (dist + eps))
    projected[:,1] = np.mod(np.arctan2(cloud[:,1], cloud[:,0]) + 2*np.pi, 2*np.pi)
    ranges[:,0] = dist
    return projected, ranges

def __convertSphericalToEuclidean(spherical):
    cart_sphere = np.zeros([len(spherical), 3])
    cart_sphere[:,0] = np.multiply(np.sin(spherical[:,0]), np.cos(spherical[:,1]))
    cart_sphere[:,1] = -np.multiply(np.sin(spherical[:,0]), np.sin(spherical[:,1]))
    cart_sphere[:,2] = -np.cos(spherical[:,0])
    mask = np.isnan(cart_sphere)
    cart_sphere[mask] = 0
    return cart_sphere

pc_sphere, ranges = __projectPointCloudOnSphere(pc)
pc_cart_sphere = __convertSphericalToEuclidean(pc_sphere)

visualize_pointcloud(pc_cart_sphere)